In [10]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
import torch
import torch.utils.data
# from torchvision import datasets, transforms
import numpy as np
import argparse
from udlp.autoencoder.denoisingAutoencoder import DenoisingAutoencoder
from pacdataset import PacDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
batch_size = 4
lr = 0.001
epochs = 2

# DEA with PAC

In [12]:
# "/Users/josephmann/Documents/Gheiratmand/sMRI competition/PAC Data/pac2018/"
datasets = {x: PacDataset(train=(x=='train'), 
#                           root_dir = "/home/paperspace/data/pac2018/") 
                          root_dir = "/Users/josephmann/Documents/Gheiratmand/sMRI competition/PAC Data/pac2018/") 
            for x in ['val','train']}

train_loader = torch.utils.data.DataLoader(
    datasets['train'],
    batch_size= batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(
    datasets['val'],
    batch_size= batch_size, shuffle=False, num_workers=2)

In [13]:
in_features = 17545
out_features = 500
dae = DenoisingAutoencoder(in_features, out_features)
dae.fit(train_loader, test_loader, lr=lr, num_epochs=epochs, loss_type="cross-entropy")

=====Denoising Autoencoding layer=======
####Epoch 0: Valid Reconstruct Loss: 12180.826
#Epoch   1: Reconstruct Loss: 3528.434, Valid Reconstruct Loss: 3671.898
#Epoch   2: Reconstruct Loss: 3341.926, Valid Reconstruct Loss: 3285.981


# SDEA with PAC

In [14]:
from udlp.autoencoder.stackedDAE import StackedDAE


In [19]:
# in_features = 784
in_features = 17545
out_features = 500
pretrainepochs = 2 

sdae = StackedDAE(input_dim=in_features, z_dim=10, binary=True,
    encodeLayer=[500,500,2000], decodeLayer=[2000,500,500], activation="relu", 
    dropout=0)
sdae.pretrain(train_loader, test_loader, lr=lr, batch_size= batch_size, 
    num_epochs=pretrainepochs, corrupt=0.3, loss_type="cross-entropy")
sdae.save_model("model/sdae.pt")
sdae.fit(train_loader, test_loader, lr= lr, num_epochs= epochs, corrupt=0.3, loss_type="cross-entropy")


l is  1
=====Denoising Autoencoding layer=======
####Epoch 0: Valid Reconstruct Loss: 12168.777
#Epoch   1: Reconstruct Loss: 3445.067, Valid Reconstruct Loss: 3420.413
#Epoch   2: Reconstruct Loss: 3294.748, Valid Reconstruct Loss: 3282.542
type(data_x) <class 'torch.FloatTensor'>
type(valid_x) <class 'torch.FloatTensor'>
l is  2
=====Denoising Autoencoding layer=======
####Epoch 0: Valid Reconstruct Loss: 1321.345
#Epoch   1: Reconstruct Loss: 116.790, Valid Reconstruct Loss: 227.774
#Epoch   2: Reconstruct Loss: 62.136, Valid Reconstruct Loss: 192.821
type(data_x) <class 'torch.FloatTensor'>
type(valid_x) <class 'torch.FloatTensor'>
l is  3
=====Denoising Autoencoding layer=======
####Epoch 0: Valid Reconstruct Loss: 822.929
#Epoch   1: Reconstruct Loss: 50.307, Valid Reconstruct Loss: 152.432
#Epoch   2: Reconstruct Loss: 26.695, Valid Reconstruct Loss: 50.826
type(data_x) <class 'torch.FloatTensor'>
type(valid_x) <class 'torch.FloatTensor'>
l is  4
=====Denoising Autoencoding laye

In [17]:
%debug

> /Users/josephmann/Documents/GitHub/UnsupervisedDeepLearning-Pytorch/udlp/autoencoder/denoisingAutoencoder.py(107)fit()
    105         total_num = 0
    106         for batch_idx, inputs in enumerate(validloader): #33
--> 107             inputs = inputs.view(inputs.size(0), -1).float()
    108             if use_cuda:
    109                 inputs = inputs.cuda()


  0.0000   3.2852   2.3741  ...    0.0000   0.0000   0.0000
  0.0000   3.2871   2.0688  ...    0.0000   0.0000   0.0000
  0.0000   2.8792   1.0228  ...    0.0000   0.0000   0.0000
           ...               ⋱              ...            
  0.0000   2.4499   1.2669  ...    0.0000   0.0000   0.0000
  0.0000   1.6096   0.0000  ...    0.0000   0.0000   0.0000
  0.0000   1.6381   2.6449  ...    0.0000   0.0000   0.0000
[torch.FloatTensor of size 538x500]


  0.0000   3.2852   2.3741  ...    0.0000   0.0000   0.0000
  0.0000   3.2871   2.0688  ...    0.0000   0.0000   0.0000
  0.0000   2.8792   1.0228  ...    0.0000   0.0000 